In [52]:
import pandas as pd
import numpy as np
from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

import sklearn.metrics as skl_metrics
import xgboost as xgb
from sklearn.cross_validation import train_test_split

import math

logger = getLogger(__name__)
pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../result_tmp/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def NWRMSLE(y, pred, weights=None):
    err2 = skl_metrics.mean_squared_log_error(y, pred, sample_weight=weights)
    return math.sqrt(err2)

def NWRMSLE_A(y, pred, weights):
    y = np.array(y)
    pred = np.array(pred)
    weights = np.array(weights)
    weighted_errors = np.dot(np.square(np.log1p(pred) - np.log1p(y)), np.transpose(weights))
    weights_sum = np.sum(weights)
    return math.sqrt(weighted_errors/weights_sum)

def NWRMSLE_lgb(pred, dtrain):
    y = list(dtrain.get_label())
    score = NWRMSLE(y, pred)
    return 'NWRMSLE', score, False

def eval_test(test_e):

    test_e['weights'] = 1
    test_e.loc[(test_e.perishable == 1), ('weights')] = 1.25

    result = NWRMSLE(test_e.unit_sales.astype(np.float64),test_e.pred_sales.astype(np.float64), test_e.weights)

    print("Eval All, Number of rows in test is", test_e.shape[0])
    print("Eval all, Forecast Period From:", min(test_e.date)," To: ", max(test_e.date))

    #### check result on first 6 days.
    test_p1 = test_e.loc[(test_e.date < '2017-08-01'), ]
    result_p1 = NWRMSLE_A(test_p1.unit_sales.astype(np.float32),test_p1.pred_sales.astype(np.float32), test_p1.weights)

    print("Eval P1, Number of rows in test is", test_p1.shape[0])
    print("Eval P1, Forecast Period From:", min(test_p1.date)," To: ", max(test_p1.date))

    #### check result on last 10 days.
    test_p2 = test_e.loc[(test_e.date >= '2017-08-01'), ]
    result_p2 = NWRMSLE_A(test_p2.unit_sales.astype(np.float32),test_p2.pred_sales.astype(np.float32), test_p2.weights)

    print("Eval P2, Number of rows in test is", test_p2.shape[0])
    print("Eval P2, Forecast Period From:", min(test_p2.date)," To: ", max(test_p2.date))

    print("Eval All Weighted NWRMSLE = ",result)
    print("Eval P1  Weighted NWRMSLE = ",result_p1)
    print("Eval P2  Weighted NWRMSLE = ",result_p2)

#--------------------------------------------------------------------------------------------------

logger.info('start')

items = pd.read_csv('../../input/items.csv'  )

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}
#train_all = pd.read_csv('../input/train.csv', usecols=[1,2,3,4,5], dtype=dtypes, parse_dates=['date'] )
train_all = pd.read_csv('../../input/train_small.csv', usecols=[1,2,3,4,5], dtype=dtypes, parse_dates=['date'] )

train_16_17 = train_all.loc[((train_all.date >= '2016-06-01') & (train_all.date <= '2016-08-31' ))
                         |((train_all.date >= '2017-06-01') & (train_all.date <= '2017-08-31' )) , ]

del train_all

#Load test
train_16_17['mset'] = 1

#Load test
#test_all = pd.read_csv('../input/test.csv', dtype=dtypes, parse_dates=['date'])
test_all = pd.read_csv('../../input/test_small.csv', dtype=dtypes, parse_dates=['date'])
test_all['mset'] = 0

df_train = pd.concat([train_16_17, test_all], ignore_index=True)

#-------------------------------------------------------------------------------------------------

df_train.loc[(df_train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
df_train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs

df_train['DOW'] = df_train['date'].dt.dayofweek
df_train['WOY'] = df_train['date'].dt.weekofyear
df_train['Year'] = df_train['date'].dt.year
df_train['Month'] = df_train['date'].dt.month
df_train['Day'] = df_train['date'].dt.day

print('training data prepared')

#-------------------------------------------------------------------------------------------------

train_m06 = df_train.loc[(df_train.Month == 6) ,]
    
ma_m06 = train_m06[['item_nbr','store_nbr','Year','unit_sales']].groupby(['item_nbr','store_nbr','Year'])['unit_sales'].mean().to_frame('avg_m06')
ma_m06.reset_index(inplace=True)

df_train = pd.merge(df_train, ma_m06, how='left', on=['item_nbr','store_nbr','Year'])

#features = (['DOW', 'WOY', 'Month', 'Day', 'avg_m06'])
#print(features)

#-------------------------------------------------------------------------------------------------

train_m07 = df_train.loc[(df_train.Month == 7) ,]
    
ma_m07 = train_m07[['item_nbr','store_nbr','Year','unit_sales']].groupby(['item_nbr','store_nbr','Year']) \
['unit_sales'].mean().to_frame('avg_m07')
ma_m07.reset_index(inplace=True)
df_train = pd.merge(df_train, ma_m07, how='left', on=['item_nbr','store_nbr','Year'])

features = (['DOW', 'WOY', 'Month', 'Day', 'avg_m06', 'avg_m07'])

#--------------------------------------------------------------------------------------------------
train = df_train.loc[(df_train.date >= '2016-08-01') & (df_train.date <= '2016-08-31' ), ]
valid = df_train.loc[(df_train.date > '2017-07-25') & (df_train.date <= '2017-08-10' ), ]
#test = df_train.loc[df_train['mset'] == 0,]

print('training data processed')



2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,636 __main__ 29 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 
2017-12-14 20:29:56,643 __main__ 84 [INFO][<module>] start 


training data prepared
training data processed


In [43]:
train.groupby(['date']).size()

date
2016-08-01    24
2016-08-02    25
2016-08-03    25
2016-08-04    25
2016-08-05    22
2016-08-06    24
2016-08-07    25
2016-08-08    26
2016-08-09    25
2016-08-10    25
2016-08-11    25
2016-08-12    26
2016-08-13    26
2016-08-14    23
2016-08-15    24
2016-08-16    24
2016-08-17    24
2016-08-18    20
2016-08-19    25
2016-08-20    25
2016-08-21    26
2016-08-22    24
2016-08-23    23
2016-08-24    22
2016-08-25    25
2016-08-26    23
2016-08-27    26
2016-08-28    24
2016-08-29    23
2016-08-30    23
2016-08-31    23
dtype: int64

In [44]:

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.3,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1301
          }
num_boost_round = 300

print("Train a XGBoost model")
X_train, X_valid = train_test_split(train, test_size=0.5, random_state=10)
y_train = np.log1p(X_train.unit_sales)
y_valid = np.log1p(X_valid.unit_sales)
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model_xgb = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=20, verbose_eval=True)


create_feature_map(features)
importance = model_xgb.get_fscore(fmap='xgb.fmap')
print(importance)



Train a XGBoost model
[0]	train-rmse:0.892159	eval-rmse:0.877252
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[1]	train-rmse:0.688105	eval-rmse:0.693599
[2]	train-rmse:0.550685	eval-rmse:0.594721
[3]	train-rmse:0.452054	eval-rmse:0.539614
[4]	train-rmse:0.381113	eval-rmse:0.498497
[5]	train-rmse:0.34845	eval-rmse:0.472817
[6]	train-rmse:0.305938	eval-rmse:0.467332
[7]	train-rmse:0.272888	eval-rmse:0.461479
[8]	train-rmse:0.236785	eval-rmse:0.467187
[9]	train-rmse:0.207402	eval-rmse:0.469224
[10]	train-rmse:0.185984	eval-rmse:0.476761
[11]	train-rmse:0.170477	eval-rmse:0.47833
[12]	train-rmse:0.157788	eval-rmse:0.485381
[13]	train-rmse:0.147169	eval-rmse:0.489382
[14]	train-rmse:0.133598	eval-rmse:0.493025
[15]	train-rmse:0.121473	eval-rmse:0.495781
[16]	train-rmse:0.110743	eval-rmse:0.500483
[17]	train-rmse:0.098547	eval-rmse:0.503705
[18]	train-rmse:0.090438	eval-rmse:0.506459
[19]	train-

In [45]:
#test.groupby(['date']).size()
#X_train[features]
train.loc[(train.store_nbr == 10) & (train.item_nbr == 103501),:]

,date,id,item_nbr,mset,onpromotion,store_nbr,unit_sales,DOW,WOY,Year,Month,Day,avg_m06,avg_m07
1515,2016-08-01,NaN,103501,1,False,10,6.0,0,31,2016,8,1,2.269231,2.740741
1539,2016-08-02,NaN,103501,1,False,10,5.0,1,31,2016,8,2,2.269231,2.740741
1563,2016-08-03,NaN,103501,1,False,10,3.0,2,31,2016,8,3,2.269231,2.740741
1589,2016-08-04,NaN,103501,1,False,10,2.0,3,31,2016,8,4,2.269231,2.740741
1614,2016-08-05,NaN,103501,1,False,10,3.0,4,31,2016,8,5,2.269231,2.740741
1636,2016-08-06,NaN,103501,1,False,10,3.0,5,31,2016,8,6,2.269231,2.740741
1660,2016-08-07,NaN,103501,1,False,10,3.0,6,31,2016,8,7,2.269231,2.740741
1685,2016-08-08,NaN,103501,1,False,10,2.0,0,32,2016,8,8,2.269231,2.740741
1711,2016-08-09,NaN,103501,1,False,10,3.0,1,32,2016,8,9,2.269231,2.740741
1736,2016-08-10,NaN,103501,1,False,10,2.0,2,32,2016,8,10,2.269231,2.740741


In [53]:
test.loc[:, 'avg_m06'].fillna(0, inplace=True) # fill NaNs
test.loc[:, 'avg_m07'].fillna(0, inplace=True) # fill NaNs

#-------------------------------------------------------------------------------------
#Load test
test = valid

test['pred_sales'] = np.exp(model_xgb.predict(xgb.DMatrix(test[features])))

test.loc[ (test['avg_m07'] == 0) | (test['avg_m06'] == 0) , 'unit_sales'] = 0


#test[['id']] = test[['id']].astype(np.int32)
#test[['id','unit_sales']].to_csv('xgb_v01.csv.gz', index=False, float_format='%.3f', compression='gzip')

#---------------------- test_e to evaluate the result --------------------------------

#test_e['error'] =  abs(test_e.pred_sales - test_e.unit_sales)
#print("Bias =",  (test_e.pred_sales.sum() - test_e.unit_sales.sum()) /  test_e.unit_sales.sum())
#print("WMAPE =",  abs(test_e.error.sum() - test_e.unit_sales.sum()) /  test_e.unit_sales.sum())


eval_test(test)

#-------------------------------------------------------------------------------------
logger.info('end')

AttributeError: 'DataFrame' object has no attribute 'perishable'

In [37]:
test.loc[(test.store_nbr == 10) & (test.item_nbr == 103501),:]

,date,id,item_nbr,mset,onpromotion,store_nbr,unit_sales,DOW,WOY,Year,Month,Day,avg_m06,avg_m07,pred_sales
4168,2017-08-16,125532151.0,103501,0,False,10,0.0,2,33,2017,8,16,2.75,2.571429,2.748190
4222,2017-08-17,125742805.0,103501,0,False,10,0.0,3,33,2017,8,17,2.75,2.571429,2.424302
4276,2017-08-18,125953459.0,103501,0,False,10,0.0,4,33,2017,8,18,2.75,2.571429,3.499528
4330,2017-08-19,126164113.0,103501,0,False,10,0.0,5,33,2017,8,19,2.75,2.571429,7.164834
4384,2017-08-20,126374767.0,103501,0,False,10,0.0,6,33,2017,8,20,2.75,2.571429,5.710910
4438,2017-08-21,126585421.0,103501,0,False,10,0.0,0,34,2017,8,21,2.75,2.571429,3.105851
4492,2017-08-22,126796075.0,103501,0,False,10,0.0,1,34,2017,8,22,2.75,2.571429,3.130021
4546,2017-08-23,127006729.0,103501,0,False,10,0.0,2,34,2017,8,23,2.75,2.571429,2.330419
4600,2017-08-24,127217383.0,103501,0,False,10,0.0,3,34,2017,8,24,2.75,2.571429,2.141097
4654,2017-08-25,127428037.0,103501,0,False,10,0.0,4,34,2017,8,25,2.75,2.571429,3.604096
